In [1]:
%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

import gtsam
import gtsam.utils.plot as gtsam_plot
import matplotlib.pyplot as plt
import numpy as np
from ipyvtklink.viewer import ViewInteractiveWidget
from vedo import  *
from pose_graph_utils import *
import tensorflow as tf
from tensorflow import  sin, cos, tan

Autosaving every 180 seconds


2025-02-24 19:26:12.899423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-24 19:26:13.417637: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/lib:
2025-02-24 19:26:13.417712: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/lib:
2025-02-24 19:26:13.417718: W tensorflow/compiler/tf2tensorrt/utils/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-02-24 19:26:14.323510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-24 19:26:14.350518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-24 19:26:14.350728: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# generate simple corkscrew trajectory

In [2]:
npts = 150
odometry_history = np.tile(np.array([0.0, 0.15, 0.08, 0.0, 0.0, 0.3]), (npts,1)) #will create corkscrew motion
# #add process noise
# odometry_history += np.random.randn(np.shape(odometry_history)[0], 
#                                     np.shape(odometry_history)[1])*np.array([0,0,0, 0.0,0.1,0.])  
# make up history of standard deviations of measurement uncertainty
pred_stds_history = np.tile(np.array([[0.001, 0.005, 0.001, 1e-6, 1e-6, 1e-5]]), (len(odometry_history),1))
ij = np.array([[0,1]])
for i in range(1,len(odometry_history)):
    ij = np.append(ij, np.array([[i, i+1]]), axis = 0)

### Try commenting out/ uncommenting the line below to add loop closure

In [3]:
ij[-1,1] = 50

In [4]:
odometry_history = np.append(odometry_history, ij, axis= 1)   #match format output by ROS package
pred_stds_history = np.append(pred_stds_history, ij, axis= 1) #match format output by ROS package

# Create an empty nonlinear factor graph
graph = gtsam.NonlinearFactorGraph() 

# Add a prior on the first pose, setting it to the origin
# A prior factor consists of a mean and a noise model (covariance matrix)
# priorMean = gtsam.Pose3(np.zeros([6,1], dtype = np.float64))  # prior at origin
priorRot = gtsam.Rot3(R_tf(np.array([0., 0., 0.8])))
priorMean = gtsam.Pose3(priorRot, np.array([0., 0., .0])) #prior at nonzero pose
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(
    np.array([0.003, 0.003, 0.003, 1e-5, 1e-5, 1e-5], dtype = np.float64))
# firstKey = initial.keys()[0] #TODO: what does this do???
graph.add(gtsam.PriorFactorPose3(0, priorMean, PRIOR_NOISE)) #constrain first point at priorMean

#loop through all constraints 
for i in range(len(ij)):
# for i in range(int(np.max(ij))):
# for i in range(1,int(np.max(ij)) + 1): #test
    #get constraint 
    raw_odom = odometry_history[i,:-2] #omit scan indices stored at end of line
    #convert to Point3 strucutre
    point = raw_odom[:3] 
    rot = gtsam.Rot3(R_tf(raw_odom[3:]))
    odometry_estimate = gtsam.Pose3(rot, point)
#     print("\n odometry estimate:\n", odometry_estimate)
    cov_estimate = gtsam.noiseModel.Diagonal.Sigmas(pred_stds_history[i,:-2])
#     cov_estimate = gtsam.noiseModel.Diagonal.Sigmas(pred_stds_history[i,:]) #dont need to omit scan indices for fake data
    first_idx = ij[i,0]
    second_idx = ij[i,1] 
#     print(first_idx, second_idx) #misses last element in ij!!!
    graph.add(gtsam.BetweenFactorPose3(first_idx, second_idx, odometry_estimate, cov_estimate))

# print(graph.size())
#set to zero initial conditions
initial = gtsam.Values()
print(initial.keys())
print("\n graph size:", graph.size())
# print(int(np.max(ij)))
# for j in range(graph.size()):
for j in range(int(np.max(ij))+1):
#     zero_rot = gtsam.Rot3(R_tf(np.array([0., 0., 0.])))
#     zero_pose3 = gtsam.Pose3(zero_rot, np.array([0., 0., 0.])) #does not work with zero initial conds??
#     initial.insert(j, zero_pose3)
    init_rot = gtsam.Rot3(R_tf(0.01*np.random.randn(3)))
    init_pose = gtsam.Pose3(init_rot, np.random.randn(3))
    initial.insert(j, init_pose)
    
# # optimize using Levenberg-Marquardt optimization
# # damped optimizer - seems to work much better here
# params = gtsam.LevenbergMarquardtParams()
# optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)

# # simple gauss newton - kinda unreliable for high DOF problems (especially with zero initial conditions)
# params = gtsam.GaussNewtonParams()
# params.setVerbosity("Termination")  # this will show info about stopping conds
# optimizer = gtsam.GaussNewtonOptimizer(graph, initial, params)

# dogleg
params = gtsam.DoglegParams()
params.setVerbosity("Termination")
params.setMaxIterations(100)
optimizer = gtsam.DoglegOptimizer(graph, initial, params)

result = optimizer.optimize()
marginals = gtsam.Marginals(graph, result) #calculate covariance estimates for each pose

print("initial error = ", graph.error(initial))
print("final error = ", graph.error(result))
# print(initial.keys())
# print(graph.keys())

2025-02-24 19:26:18.691923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-24 19:26:18.692680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-24 19:26:18.692890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-24 19:26:18.693057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

KeyVector[]

 graph size: 151
initial error = converged
 325748091652201.06
final error =  19122.04429112809
errorThreshold: 19122 <? 0
absoluteDecrease: 0.0290529541198 <? 1e-05
relativeDecrease: 1.51934121353e-06 <? 1e-05
iterations: 45 >? 100


In [5]:
#Plot results
plt1 = Plotter(N = 1, axes = 1, bg =(1, 1, 1), interactive = True) #ax=7 gives rulers
disp = []
# disp = plot_results(disp, result, ij, draw_axis=True) #draw without ellipsoids
disp = plot_results(disp, result, ij, marginals, draw_axis = True) #draws covarince ellipsoids
plt1.show(disp, "Factor Graph Test")

 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_seq

 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_sequence()
 Warning! isSequence() is deprecated. Please use is_seq